In [1]:
'''
Available methods are the followings:
[1] StepwiseRegression

Authors: Danusorn Sitdhirasdr <danusorn.si@gmail.com>
versionadded:: 31-01-2024

'''
import pandas as pd, numpy as np, os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (r2_score, mean_squared_error, roc_auc_score)
from itertools import product
from scipy import stats

__all__ = ["StepwiseRegression"]

In [2]:
class ValidateParams:
    
    '''Validate parameters'''
    
    def Interval(self, Param, Value, dtype=int, 
                 left=None, right=None, closed="both"):

        '''
        Validate numerical input.

        Parameters
        ----------
        Param : str
            Parameter's name

        Value : float or int
            Parameter's value

        dtype : {int, float}, default=int
            The type of input.

        left : float or int or None, default=None
            The left bound of the interval. None means left bound is -∞.

        right : float, int or None, default=None
            The right bound of the interval. None means right bound is +∞.

        closed : {"left", "right", "both", "neither"}
            Whether the interval is open or closed. Possible choices are:
            - "left": the interval is closed on the left and open on the 
              right. It is equivalent to the interval [ left, right ).
            - "right": the interval is closed on the right and open on the 
              left. It is equivalent to the interval ( left, right ].
            - "both": the interval is closed.
              It is equivalent to the interval [ left, right ].
            - "neither": the interval is open.
              It is equivalent to the interval ( left, right ).

        Returns
        -------
        Value : float or int
            Parameter's value

        '''
        Options = {"left"    : (np.greater_equal, np.less), # a<=x<b
                   "right"   : (np.greater, np.less_equal), # a<x<=b
                   "both"    : (np.greater_equal, np.less_equal), # a<=x<=b
                   "neither" : (np.greater, np.less)} # a<x<b

        f0, f1 = Options[closed]
        c0 = "[" if f0.__name__.find("eq")>-1 else "(" 
        c1 = "]" if f1.__name__.find("eq")>-1 else ")"
        v0 = "-∞" if left is None else str(dtype(left))
        v1 = "+∞" if right is None else str(dtype(right))
        if left  is None: left  = -np.inf
        if right is None: right = +np.inf
        interval = ", ".join([c0+v0, v1+c1])
        tuples = (Param, dtype.__name__, interval, Value)
        err_msg = "%s must be %s or in %s, got %s " % tuples    

        if isinstance(Value, dtype):
            if not (f0(Value, left) & f1(Value, right)):
                raise ValueError(err_msg)
        else: raise ValueError(err_msg)
        return Value

    def StrOptions(self, Param, Value, options, dtype=str):

        '''
        Validate string or boolean inputs.

        Parameters
        ----------
        Param : str
            Parameter's name
            
        Value : float or int
            Parameter's value

        options : set of str
            The set of valid strings.

        dtype : {str, bool}, default=str
            The type of input.
        
        Returns
        -------
        Value : float or int
            Parameter's value

        '''
        if Value not in options:
            err_msg = f'{Param} ({dtype.__name__}) must be either '
            for n,s in enumerate(options):
                if n<len(options)-1: err_msg += f'"{s}", '
                else: err_msg += f' or "{s}" , got %s'
            raise ValueError(err_msg % Value)
        return Value
    
    def check_range(self, param0, param1):
        
        '''
        Validate number range.
        
        Parameters
        ----------
        param0 : tuple(str, float)
            A lower bound parameter e.g. ("name", -100.)
            
        param1 : tuple(str, float)
            An upper bound parameter e.g. ("name", 100.)
        '''
        if param0[1] >= param1[1]:
            raise ValueError(f"`{param0[0]}` ({param0[1]}) must be less"
                             f" than `{param1[0]}` ({param1[1]}).")

In [202]:
class ANOVA:
    
    def __init__(self, alpha=0.05):
        
        '''
        Parameters
        ----------
        alpha : float, default=0.05
            It refers to the likelihood that the population lies outside 
            the confidence interval (two-tailed distribution).
        
        '''
        self.alpha = alpha
    
    def __linear__(self, X, y_true, y_pred, coefs):
        
        '''
        Analysis of variance for linear regression
        
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data.
        
        y_true : array-like of shape (n_samples,)
            Ground truth (correct) target values.

        y_pred : array-like of shape (n_samples,)
            Estimated target values.
            
        coefs : array of shape (intercept_ + n_features,)
            Estimated coefficients for the linear regression problem. 
        
        Returns
        -------
        {"coef"   : estimated coefficients
         "stderr" : standard errors
         "t"      : t-statistics
         "pvalue" : P-values (two-tailed distribution)
         "lower"  : lower bounds
         "upper"  : upper bounds, 
         "r2"     : R-Squared
         "adj_r2" : adjusted R-Squared
         "mse"    : Mean-Squared-Error}
         
        '''
        # Initialize parameters
        X = np.array(X).copy()
        X = np.hstack((np.ones((len(X),1)), X))
        y_true = np.array(y_true).flatten()
        y_pred = np.array(y_pred).flatten()
        coefs  = coefs.flatten()
        
        # Degree of freedom and MSE
        N,K = X.shape
        df  = N - K - 1
        mse = (sum((y_true-y_pred)**2)) / df
        
        # Standard error, t-stat, and p-values
        stderrs  = np.sqrt(mse*(np.linalg.inv(X.T.dot(X)).diagonal()))
        t_stats  = coefs / stderrs
        p_values = np.array([2*(1-stats.t.cdf(abs(t), df)) for t in t_stats])
        t = stats.t.ppf(1-self.alpha/2, df) * np.abs(stderrs)
        
        # Regression statistics
        r2 = r2_score(y_true, y_pred)
        factor = ((1-r2) * ((N-1)/(N-K-1)))
        adj_r2 = r2 - (1-r2) * factor
        mse = (sum((y_true-y_pred)**2)) / (N-K-1)

        return {"coef"   : coefs,
                "stderr" : stderrs, 
                "t"      : t_stats, 
                "pvalue" : p_values, 
                "lower"  : coefs - t, 
                "upper"  : coefs + t, 
                "r2"     : r2, 
                "adj_r2" : adj_r2, 
                "mse"    : mse}
    
    def __logistic__(self, X, y_true, y_score, coefs):
        
        '''
        =================================================================
        Analysis of variance for linear regression
        
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data.
        
        y_true : array-like of shape (n_samples,)
            True labels or binary label indicators. The binary and multiclass 
            cases expect labels with shape (n_samples,) while the multilabel 
            case expects binary label indicators with shape (n_samples, n_classes).
        
        
        
        
        
        y_score array-like of shape (n_samples,) or (n_samples, n_classes)
            Target scores. In the binary case, it corresponds to an array 
            of shape (n_samples,). Both probability estimates and non-thresholded 
            decision values can be provided. The probability estimates correspond 
            to the probability of the class with the greater label, i.e. 
            estimator.classes_[1] and thus estimator.predict_proba(X, y)[:, 1]. 
            The decision values corresponds to the output of 
            estimator.decision_function(X, y). See more information in the User guide;
        
        y_proba : array-like of shape (n_samples,)
            Estimated target values.
            
        coefs : array of shape (intercept_ + n_features,)
            Estimated coefficients for the linear regression problem. 
        
        Returns
        -------
        {"coef"   : estimated coefficients
         "stderr" : standard errors
         "t"      : t-statistics
         "pvalue" : P-values (two-tailed distribution)
         "lower"  : lower bounds
         "upper"  : upper bounds, 
         "gini"   : GINI index
         "ks"     : adjusted R-Squared}
         
        '''
        # Initialize parameters
        X = np.array(X).copy()
        X = np.hstack((np.ones((len(X),1)), X))
        y_true = np.array(y_true).flatten()
        y_score= np.array(y_score).flatten()
        coefs  = coefs.flatten()
        
        # Degree of freedom and MSE
        N,K = X.shape
        df  = N - K - 1
        mse = (sum((y_true-y_pred)**2)) / df
        
        # Standard error, t-stat, and p-values
        stderrs  = np.sqrt(mse*(np.linalg.inv(X.T.dot(X)).diagonal()))
        t_stats  = coefs / stderrs
        p_values = np.array([2*(1-stats.t.cdf(abs(t), df)) for t in t_stats])
        t = stats.t.ppf(1-self.alpha, df) * np.abs(stderrs)
        
        
        W = np.identity(len(y_pred)) * y_pred*(1-y_pred)
        stderr = np.sqrt(np.linalg.inv(aX.T.dot(W).dot(aX)).diagonal())
        z = abs(np.r_[clf.coef_.flatten(), clf.intercept_])/stderr
        
        
        # Regression statistics
        auc = roc_auc_score(y_true, y_score)
      
        return {"coef"   : coefs,
                "stderr" : stderrs, 
                "t"      : t_stats, 
                "pvalue" : p_values, 
                "lower"  : coefs - t, 
                "upper"  : coefs + t, 
                "gini"   : 2*auc-1, 
                "ks"     : adj_r2}

In [203]:
class StepwiseRegression(ANOVA, ValidateParams):
    
    '''
    Stepwise Regression

    Parameters
    ----------
    estimator : Estimator instance, default=None
        A supervised learning estimator with `fit` method that 
        provides attribute about feature importance i.e. `coef_`.
        If None, it defaults to LinearRegression (sklearn).

    method : {"backward", "forward", "stepwise"}, default="forward"
        - 'forward'  : forward selection
        - 'backward' : backward elimination
        - 'stepwise' : a combination of forward selection and backward
                       elimination

    alpha : float, default=0.05
        Passing criterion or p-value of two-tailed distribution.
    
    Attributes
    ----------
    estimator_ : Estimator instance
        Fitted estimator with selected features.
    
    features : list of str
        List of selected features.
    
    results_ : dict of numpy (masked) ndarrays
        It contains results from each step of selection/elimination.

    '''
    def __init__(self, estimator=None, method='forward', alpha=0.05):
        
        # Check estimator
        if estimator is None: estimator = LinearRegression()
        if (hasattr(estimator, "fit") + hasattr(estimator, "predict")) < 2:
            raise ValueError('An estimator must have `fit`and `predict` '
                             'methods and provides feature importance '
                             'i.e. coef_.')
        else: self.estimator = estimator
        
        args = ('method', method, ["forward", "backward", "stepwise"], str)
        self.method = self.StrOptions(*args)
        super().__init__(alpha)
    
    def fit(self, X, y, sample_weight=None, n_features=None, use_features=None):
        
        '''
        Fit linear model.
        
        Parameters
        ----------
        X : pd.DataFrame of shape (n_samples, n_features)
            Training data.

        y : array-like of shape (n_samples,)
            Target values.
            
        sample_weight : array-like of shape (n_samples,), default=None
            Array of weights that are assigned to individual samples. If 
            not provided, then each sample is given unit weight.
        
        n_features : int, default=None
            The number of features to be retained. If None, it defaults 
            to X.shape[1] when `method` is "forward", and 1 when `method` 
            is "backward".

        use_features : list of str, default=None
            Subset of features in `X` to start with. All elements must be 
            strings that correspond to column names. If None, no features 
            is selected. This is only relevant when `method` is "forward".
        
        Returns
        -------
        self : object
            Returns the instance itself.
            
        '''
        # Validate parameters
        if isinstance(X, pd.DataFrame)==False: 
            raise ValueError(f"`X` must be pandas.DataFrame. " 
                             f"Got {type(X)} instead.")
        
        # Initialize parameters
        self.features = list(X)
        self.results_ = dict()
        
        args = (X, y, sample_weight, n_features, use_features)
        if self.method == "backward":
            self.features = self.__backward__(*args[:4])
        elif self.method == "forward":
            self.features = self.__forward__(*args)
        elif self.method == "stepwise":
            self.features = self.__stepwise__(*args[:3])
        else: pass

        # Fit estimator with final set of features
        args = (X[self.features], y, sample_weight)
        self.estimator_ = self.estimator.fit(*args)
        
        return self
    
    def __backward__(self, X, y, sample_weight=None, n_features=None):
        
        '''
        Backward elimination removes features from the full estimator one
        by one until no features overcome the threshold value i.e. greater
        than defined p-value. Features never return once removed.
        
        Parameters
        ----------
        X : pd.DataFrame of shape (n_samples, n_features)
            Training data.

        y : array-like of shape (n_samples,)
            Target values.

        sample_weight : array-like of shape (n_samples,), default=None
            Array of weights that are assigned to individual samples. If 
            not provided, then each sample is given unit weight.
            
        n_features : int, default=None
            The number of features to be retained in estimator. If None, 
            it defaults to 1.
            
        Returns
        -------
        use_features : list of str
            List of remaining features.
            
        '''
        # Initialize parameters
        use_features = list(X)
        if n_features is None: n_features = 1 
        
        # Validate parameters
        args = (int, 1, X.shape[1], "both")
        self.Interval("n_features", n_features, *args)
        
        while len(use_features) > n_features:
            
            # Fit estimator and perform ANOVA
            anova = self.__anova__(X[use_features].copy(), y, sample_weight)
            self.results_.update({len(self.results_) : anova})
            
            # Determine variable with maximum p-value
            k = np.argmax(anova["pvalue"])
            if anova["pvalue"][k] < self.alpha: break
            else: use_features.remove(use_features[k])

        return use_features
    
    def __forward__(self, X, y, sample_weight=None, n_features=None, use_features=None):
        
        '''
        Forward selection adds features one by one to an empty estimator
        until no features overcome the threshold value i.e. less than or
        equal to defined p-value. Features never leave once added.
        
        Parameters
        ----------
        X : pd.DataFrame of shape (n_samples, n_features)
            Training data.

        y : array-like of shape (n_samples,)
            Target values. 

        sample_weight : array-like of shape (n_samples,), default=None
            Array of weights that are assigned to individual samples. If 
            not provided, then each sample is given unit weight.
            
        n_features : int, default=None
            The number of features to be retained. If None, it defaults 
            to maximum number of features i.e. X.shape[1].
            
        use_features : list of str, default=None
            Subset of features in `X` to start with. All elements must be 
            strings that correspond to column names. If None, no features 
            is selected.

        Returns
        -------
        use_features : list of str
            List of remaining features.
            
        '''
        # Initialize parameters
        if use_features is None: use_features =list()
        features = list(set(list(X)).difference(use_features))
        if n_features is None: n_features = X.shape[1]
        
        # Validate parameters 
        args = (int, 1, X.shape[1], "both")
        self.Interval("n_features", n_features, *args)
        
        while len(use_features) < n_features:
            
            min_pvalue = 1.0
            for new_feature in features:
                
                # Fit estimator and perform ANOVA
                X0 = X[use_features + [new_feature]].copy()
                anova  = self.__anova__(X0, y, sample_weight)
                pvalue = anova["pvalue"][-1]
                
                # Find next best feature
                if pvalue < min_pvalue:
                    min_pvalue   = pvalue
                    best_feature = new_feature
                    best_stderrs = anova
                
            # Determine variable with minimum p-value
            if min_pvalue <= self.alpha:
                use_features.append(best_feature)
                features.remove(best_feature)
                self.results_.update({len(self.results_) : best_stderrs})
            else: break

        return use_features
    
    def __stepwise__(self, X, y, sample_weight=None):
        
        '''
        Stepwise regression is a combination of forward selection and
        backward elimination. At each step, a new feature that satisfies
        criterion i.e. p-value is added. Then a model gets evaluated. If 
        one or more features are no longer passing p-value, they are 
        pruned. Then the process repeats until set of features does not 
        change.
        
        Parameters
        ----------
        X : pd.DataFrame of shape (n_samples, n_features)
            Training data.

        y : array-like of shape (n_samples,)
            Target values.

        sample_weight : array-like of shape (n_samples,), default=None
            Array of weights that are assigned to individual samples. If 
            not provided, then each sample is given unit weight.
        
        Returns
        -------
        use_features : list of str
            List of remaining features.
            
        '''
        use_features, base_features = list(), [None]
        while set(base_features)!=set(use_features):
            base_features = use_features.copy()
            args = (X, y, sample_weight, len(use_features)+1, use_features)
            use_features = self.__forward__(*args)
            use_features = self.__backward__(X[use_features], y, sample_weight)

        return use_features
    
    def __anova__(self, X, y, sample_weight=None):
        
        '''Fit estimator and perform ANOVA'''
        model = self.estimator.fit(X, y, sample_weight)
        return self.__linear__(X, y, model.predict(X), 
                               np.r_[model.intercept_,
                                     model.coef_.flatten()])

## Linear Regression

In [65]:
from sklearn.datasets import make_regression

In [198]:
X, y, coef = make_regression(n_samples=1000, n_features=50, n_informative=11, 
                             bias=0.2, noise=0.2, shuffle=True, coef=True, 
                             random_state=0)

In [199]:
X = pd.DataFrame(X, columns=["V{}".format(str(n).zfill(2)) for n in range(1,X.shape[1]+1)])

In [200]:
X.head()

V01       V02       V03       V04       V05       V06       V07  \
0 -0.426673 -0.765382  1.274088  0.062721  0.640408 -0.321763 -1.335802   
1  1.222251  0.610705  0.323958  1.553818  1.143920  0.882683 -1.635516   
2 -1.093653 -0.270213 -0.541143 -0.868016  0.354762 -0.523756  0.141621   
3  0.785005  1.133185 -0.499190 -1.630697  0.573077  0.788635  1.250590   
4  0.243892  0.170598 -1.047781 -0.947740 -0.969112 -0.865302  0.796561   

        V08       V09       V10  ...       V41       V42       V43       V44  \
0 -1.320190  0.097716 -0.901783  ...  1.170697 -2.182884 -0.311977 -1.016176   
1  0.134273  0.840998 -0.796787  ... -0.994970 -0.886952 -1.243777 -1.180108   
2 -0.582613  0.340076 -0.323999  ... -0.432157 -0.200931  0.217588 -0.301530   
3 -1.471126  0.248429 -0.647782  ...  0.534387 -1.512426 -0.216212 -1.625108   
4  1.526307 -0.092528 -2.106197  ... -0.562252 -1.570655 -1.938357  0.133180   

        V45       V46       V47       V48       V49       V50  
0  0.523206  0.898704  0.110078  0.088458 -0.249263 -0.287137  
1  0.897069 -0.263733 -0.544241 -0.855114 -0.695952  0.248069  
2 -2.154650  0.590571  1.535657 -0.467766  0.850616  0.355597  
3  0.577062 -2.010642 -2.629108 -0.889607 -1.609317  0.126931  
4  0.283647 -1.111379 -0.939641  0.475027  0.859331  0.045522  

[5 rows x 50 columns]

In [204]:
mm = StepwiseRegression(estimator=LinearRegression(fit_intercept=False),alpha=0.01, method="forward").fit(X, y)
print(mm.features)
r2 = [mm.results_[key]["r2"] for key in mm.results_.keys()]
print(r2)

['V47', 'V29', 'V14', 'V01', 'V27', 'V20', 'V25', 'V38', 'V35', 'V50', 'V48']
[0.20554618236968147, 0.3711604677318293, 0.42520554294488055, 0.4661905019303382, 0.5035995216601452, 0.5560689103301661, 0.8159272181173294, 0.8353698548172894, 0.8586438448976752, 0.997166993883598, 0.999998022789351]


In [205]:
mm.results_[10]

{'coef': array([ 0.        , 98.67455971, 86.5640707 , 50.94852102, 44.89934065,
        37.46559664, 41.77877161, 99.66797808, 30.39939489, 27.70762382,
        74.22689559, 10.63493748]),
 'stderr': array([0.00895321, 0.00881867, 0.00907322, 0.00907167, 0.00911152,
        0.00908758, 0.00869009, 0.00901127, 0.00916368, 0.0088034 ,
        0.00890302, 0.008947  ]),
 't': array([    0.        , 11189.27738813,  9540.6085685 ,  5616.2253062 ,
         4927.757355  ,  4122.7254226 ,  4807.6312154 , 11060.36370218,
         3317.37817551,  3147.37620489,  8337.27576439,  1188.6596318 ]),
 'pvalue': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'lower': array([-2.08621561e-02,  9.86540110e+01,  8.65429289e+01,  5.09273828e+01,
         4.48781096e+01,  3.74444214e+01,  4.17585225e+01,  9.96469806e+01,
         3.03780423e+01,  2.76871107e+01,  7.42061504e+01,  1.06140898e+01]),
 'upper': array([2.08621561e-02, 9.86951084e+01, 8.65852125e+01, 5.09696592e+01,
        4.49205717e

In [84]:
kk = X[['V47', 'V29', 'V14', 'V01', 'V27', 'V20', 'V25', 'V38', 'V35', 'V50', 'V48']].copy()
kk["Intercept"] = 1
model = sm.OLS(y,kk)
results = model.fit(fit_intercept=True)

In [85]:
print(results.summary()) 

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 9.114e+07
Date:                Fri, 05 Jan 2024   Prob (F-statistic):               0.00
Time:                        10:33:00   Log-Likelihood:                 203.99
No. Observations:                1000   AIC:                            -384.0
Df Residuals:                     988   BIC:                            -325.1
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
V47           98.6773      0.006   1.59e+04      0.0

## Logistics

In [49]:
from sklearn.datasets import load_iris

In [134]:
a = load_iris()
X = pd.DataFrame(a.data, columns=a.feature_names)
X["intercept"] = 1
y = (a.target==1).astype(int)

In [135]:
log_reg = sm.Logit(y, X).fit(maxiter=200, method='lbfgs')

In [114]:
# printing the summary table 
print(log_reg.summary()) 

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  150
Model:                          Logit   Df Residuals:                      145
Method:                           MLE   Df Model:                            4
Date:                Fri, 05 Jan 2024   Pseudo R-squ.:                  0.2403
Time:                        10:51:25   Log-Likelihood:                -72.535
converged:                       True   LL-Null:                       -95.477
Covariance Type:            nonrobust   LLR p-value:                 2.603e-09
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
sepal length (cm)    -0.2454      0.650     -0.378      0.706      -1.518       1.028
sepal width (cm)     -2.7966      0.784     -3.569      0.000      -4.332      -1.261
petal length (cm)     1.

In [166]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=100, fit_intercept=True, solver="lbfgs")
clf.fit(X.drop(columns="intercept"), y)

LogisticRegression(random_state=100)

In [174]:
type(LinearRegression())

sklearn.linear_model._base.LinearRegression

In [177]:
# dir(LogisticRegression()) predict_proba

In [167]:
clf.coef_

array([[-0.1792787 , -2.12866718,  0.69665417, -1.27480129]])

In [168]:
clf.intercept_

array([5.58615272])

In [152]:
y_pred = clf.predict_proba(X.drop(columns="intercept"))[:,[1]]
aX = X.drop(columns="intercept").values.copy()
aX = np.hstack((aX,np.ones((len(aX),1))))

In [171]:
W = np.identity(len(y_pred)) * y_pred*(1-y_pred)
stderr = np.sqrt(np.linalg.inv(aX.T.dot(W).dot(aX)).diagonal())
stderr = abs(np.r_[clf.coef_.flatten(), clf.intercept_])/stderr

array([0.60369273, 0.69503008, 0.60883153, 1.01166487, 2.27940944])

In [172]:
abs(np.r_[clf.coef_.flatten(), clf.intercept_])/stderr

array([0.29697011, 3.06269793, 1.14424786, 1.26010236, 2.45070175])